In [1]:
pip install psycopg2

In [2]:
import os
import psycopg2
import pandas as pd
import datetime
from datetime import date

# today's date
today = date.today()

# Define the start and end dates
start_date = '2024-07-01'
end_date = today.strftime('%Y-%m-%d')

# Connection details for the first database
conn1_details = {
    'dbname': 'trips',
    'user': 'postgres',
    'password': 'theimm0rtaL',
    'host': '34.100.223.97',
    'port': '5432'
}

# Connection details for the second database (master table)
conn2_details = {
    'dbname': 'master_prod',
    'user': 'postgres',
    'password': 'theimm0rtaL',
    'host': '34.100.223.97',
    'port': '5432'
}

def connect_to_database(conn_details):
    try:
        conn = psycopg2.connect(**conn_details)
        print('Connection Created')
        return conn
    except psycopg2.DatabaseError as e:
        print('Database Error:')
        print(str(e))
        return None
    except psycopg2.Error as e:
        print('Connection Error:')
        print(str(e))
        return None

# Query for the second database
query1 = f"""
    SELECT
    site_name,
    reg_number,
    new_mapped_status,
    record_date,
    COALESCE(
        CASE
            WHEN site_name = 'ADPHYD-HYD-NANAKARAMGUDA-TRIP' AND new_mapped_status = 'SITE' THEN 'Trip'
            WHEN site_name = 'BirlasoftPNQ-PNQ-Birlasoft Pune' AND new_mapped_status = 'SITE' THEN 'DEDICATED_24'
            WHEN site_name = 'DuPontHYD-HYD-Dupont_Waverock' AND new_mapped_status = 'SITE' THEN 'Trip'
            WHEN site_name = 'FISERV-PNQ-FISERV' AND new_mapped_status = 'SITE' THEN 'Trip'
            WHEN site_name = 'GenpactHYD-HYD-Genpact_HYD' AND new_mapped_status = 'SITE' THEN 'DEDICATED_12'
            WHEN site_name = 'GoogleHYD-HYD-Salarpuriya Sattva' AND new_mapped_status = 'SITE' THEN 'DEDICATED_24'
            WHEN site_name = 'Sagility-HYD-Purva Summit Hitec City-D24' AND new_mapped_status = 'SITE' THEN 'DEDICATED_24'
            ELSE new_mapped_status_subtype
        END,
        'not_used'
    ) AS Correct_Map_Status,
    COALESCE(
        CASE
            WHEN site_name = 'GoogleNCR-NCR-Google' AND new_mapped_status = 'SITE' THEN 'NCR-GGN'
            WHEN site_name = 'ICICIIDR-IDR-ICICIIDR' AND new_mapped_status = 'SITE' THEN 'IDR'
            WHEN site_name = 'KPMGNCR-NCR-KPMG_DLF_Cybercity' AND new_mapped_status = 'SITE' THEN 'NCR-GGN'
            WHEN site_name = 'Mumbai-Spot' AND new_mapped_status = 'SITE' THEN 'MUM'
            WHEN site_name = 'TelstraPune-Pune-QBP' AND new_mapped_status = 'SITE' THEN 'PNQ'
            ELSE operating_city
        END,
        'not_used'
    ) AS operating_city
FROM
    vehicle_site_mapping_daily_history
WHERE
    record_date BETWEEN '{start_date}' AND '{end_date}' AND new_mapped_status = 'SITE';
;
"""

# Query for the first database
query2 = f"""
    WITH cte1 AS (
    SELECT
        CLIENT,
        CITY_CODE,
        CLIENT_OFFICE,
        TRIP_DATE,
        '0' as model,
        0 AS package_hour,
        COUNT(*) AS TRIP_COUNT,
        COUNT(CASE WHEN IS_ESCORT = '1' THEN 1 END) AS ESCORT_COUNT,
        Billing_model,
        'ICE' AS cet_models
    FROM public.ice_etms_trips
    WHERE trip_date BETWEEN '{start_date}' AND '{end_date}'
    GROUP BY CLIENT, Billing_model, CITY_CODE, CLIENT_OFFICE, TRIP_DATE, cet_models, package_hour
),
cte2 AS (
    SELECT
        CLIENT,
        CITY_CODE,
        CLIENT_OFFICE,
        TRIP_DATE,
        '0' as model,
        0 AS package_hour,
        COUNT(*) AS TRIP_COUNT,
        COUNT(CASE WHEN IS_ESCORT = '1' THEN 1 END) AS ESCORT_COUNT,
        Billing_model,
        'CET' AS cet_models
    FROM PUBLIC.ETMS_TRIPS
    WHERE trip_date BETWEEN '{start_date}' AND '{end_date}'
    GROUP BY CLIENT, Billing_model, CITY_CODE, CLIENT_OFFICE, TRIP_DATE, cet_models,package_hour
),
cte3 AS (
    SELECT
        site_name AS CLIENT,
        CITY_CODE,
        site_name AS CLIENT_OFFICE,
        TRIP_DATE,
        model,
        COUNT(*) AS TRIP_COUNT,
        0 AS ESCORT_COUNT,
        'Spot' AS Billing_model,
        Package_hour,
        'Spot' AS cet_models
    FROM public.etms_spot_trips
    WHERE trip_date BETWEEN '{start_date}' AND '{end_date}'
    GROUP BY CLIENT,Billing_model,  CITY_CODE, CLIENT_OFFICE, TRIP_DATE, cet_models, model, Package_hour
)
SELECT
    CLIENT,
    Billing_model,
    CITY_CODE,
    CLIENT_OFFICE,
    TRIP_DATE,
    TRIP_COUNT,
    model,
    package_hour,
    ESCORT_COUNT,
    cet_models
FROM cte1
UNION ALL
SELECT
    CLIENT,
    Billing_model,
    CITY_CODE,
    CLIENT_OFFICE,
    TRIP_DATE,
    TRIP_COUNT,
    model,
    package_hour,
    ESCORT_COUNT,
    cet_models
FROM cte2
UNION ALL
SELECT
    CLIENT,
    Billing_model,
    CITY_CODE,
    CLIENT_OFFICE,
    TRIP_DATE,
    TRIP_COUNT,
    model,
    package_hour,
    ESCORT_COUNT,
    cet_models
FROM cte3;
"""

# Execute the first query and store the result in df1
conn1 = connect_to_database(conn2_details)
if conn1:
    cursor1 = conn1.cursor()
    cursor1.execute(query1)

    # Retrieve Dataset from SQL Server
    recordset1 = cursor1.fetchall()

    columns1 = [desc[0] for desc in cursor1.description]

    # Convert to DataFrame
    df1 = pd.DataFrame(recordset1, columns=columns1)

    # Convert all date and datetime columns to string and handle NaN values
    for column in df1.columns:
        if pd.api.types.is_datetime64_any_dtype(df1[column]) or pd.api.types.is_timedelta64_dtype(df1[column]) or pd.api.types.is_period_dtype(df1[column]):
            df1[column] = df1[column].astype(str)
        elif pd.api.types.is_object_dtype(df1[column]):
            df1[column] = df1[column].apply(lambda x: str(x) if isinstance(x, (datetime.date, datetime.datetime)) else x)

    # Replace NaN values with empty strings
    df1 = df1.fillna('')

    cursor1.close()
    conn1.close()

# Execute the second query and store the result in df2
conn2 = connect_to_database(conn1_details)
if conn2:
    cursor2 = conn2.cursor()
    cursor2.execute(query2)

    # Retrieve Dataset from SQL Server
    recordset2 = cursor2.fetchall()

    columns2 = [desc[0] for desc in cursor2.description]

    # Convert to DataFrame
    df2 = pd.DataFrame(recordset2, columns=columns2)

    # Convert all date and datetime columns to string and handle NaN values
    for column in df2.columns:
        if pd.api.types.is_datetime64_any_dtype(df2[column]) or pd.api.types.is_timedelta64_dtype(df2[column]) or pd.api.types.is_period_dtype(df2[column]):
            df2[column] = df2[column].astype(str)
        elif pd.api.types.is_object_dtype(df2[column]):
            df2[column] = df2[column].apply(lambda x: str(x) if isinstance(x, (datetime.date, datetime.datetime)) else x)

    # Replace NaN values with empty strings
    df2 = df2.fillna('')

    cursor2.close()
    conn2.close()


Connection Created


<ipython-input-2-c3256909b679>:190: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_datetime64_any_dtype(df1[column]) or pd.api.types.is_timedelta64_dtype(df1[column]) or pd.api.types.is_period_dtype(df1[column]):
<ipython-input-2-c3256909b679>:190: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_datetime64_any_dtype(df1[column]) or pd.api.types.is_timedelta64_dtype(df1[column]) or pd.api.types.is_period_dtype(df1[column]):


Connection Created


<ipython-input-2-c3256909b679>:217: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_datetime64_any_dtype(df2[column]) or pd.api.types.is_timedelta64_dtype(df2[column]) or pd.api.types.is_period_dtype(df2[column]):


In [3]:
df1['site_name']=df1['site_name'].str.upper()
df2['client_office']=df2['client_office'].str.upper()
df1['correct_map_status']=df1['correct_map_status'].str.upper()
df2['billing_model']=df2['billing_model'].str.upper()

In [4]:
df1

,site_name,reg_number,new_mapped_status,record_date,correct_map_status,operating_city
0,ACCENTURENCR-NCR-UNITECH INFOSPACE-D24,DL52GD4098,SITE,2024-07-26,DEDICATED_24,NCR-GGN
1,HCL-JIGANI SPECIAL ECONOMIC ZONE-BLR,KA03AK7155,SITE,2024-07-26,TRIP,BLR
2,HCL-JIGANI SPECIAL ECONOMIC ZONE-BLR,KA03AK8270,SITE,2024-07-26,TRIP,BLR
3,ACCENTURENCR-NCR-UNITECH INFOSPACE-D24,DL52GD4041,SITE,2024-07-26,DEDICATED_24,NCR-GGN
4,LITBLR-BLR-OFFICE,KA51AJ3030,SITE,2024-07-26,SPOT,BLR
...,...,...,...,...,...,...
63015,ADPPUN-PUN-VIMANNAGAR-TRIP,MH12VF6036,SITE,2024-08-13,TRIP,PNQ
63016,TESCOBLR-BLR-TESCOCAMPUS,KA03AK9445,SITE,2024-08-13,TRIP,BLR
63017,VODAFONEPNQ-PNQ-VODAFONE EON,MH12VF9853,SITE,2024-08-13,TRIP,PNQ
63018,WELLSFARGOHYD-HYD-HITECH CITY-D12,TS07UM1900,SITE,2024-08-13,DEDICATED_12,HYD


In [5]:
# Group by the specified columns and count the 'reg_number'
df1 = df1.groupby(['site_name', 'record_date', 'operating_city', 'correct_map_status'])['reg_number'].count()

# Reset the index to convert grouped data back into a DataFrame
df1 = df1.reset_index()

In [6]:
#df1 if the correct_map_status=not_used
df1=df1[df1['correct_map_status']!='NOT_USED']

In [7]:
df1

,site_name,record_date,operating_city,correct_map_status,reg_number
0,24BY7BLR-BLR-24BY7 PRESTIGE TECH PARK,2024-07-01,BLR,DEDICATED_24,16
1,24BY7BLR-BLR-24BY7 PRESTIGE TECH PARK,2024-07-02,BLR,DEDICATED_24,15
2,24BY7BLR-BLR-24BY7 PRESTIGE TECH PARK,2024-07-03,BLR,DEDICATED_24,15
4,24BY7BLR-BLR-24BY7 PRESTIGE TECH PARK,2024-07-04,BLR,DEDICATED_24,15
6,24BY7BLR-BLR-24BY7 PRESTIGE TECH PARK,2024-07-05,BLR,DEDICATED_24,17
...,...,...,...,...,...
5498,ZSPUNE-PNQ-ZS PUNE,2024-08-09,PNQ,DEDICATED_24,10
5499,ZSPUNE-PNQ-ZS PUNE,2024-08-10,PNQ,DEDICATED_24,9
5500,ZSPUNE-PNQ-ZS PUNE,2024-08-11,PNQ,DEDICATED_24,9
5501,ZSPUNE-PNQ-ZS PUNE,2024-08-12,PNQ,DEDICATED_24,9


In [8]:
df1.columns

Index(['site_name', 'record_date', 'operating_city', 'correct_map_status',
       'reg_number'],
      dtype='object')

In [9]:
# Merge the dataframes
merged_df = pd.merge(df2,df1,
                     left_on=['client_office', 'trip_date','billing_model'],
                     right_on=['site_name', 'record_date','correct_map_status'],
                     how='left')

# Rename the 'reg_number' column from df1 to 'vehicle_count_from_df3'
merged_df = merged_df.rename(columns={'reg_number': 'Vehicle Count'})

# Select the desired columns
df4 = merged_df[[ 'client','client_office', 'billing_model', 'city_code','trip_date', 'trip_count',
                 'escort_count', 'model', 'package_hour', 'cet_models',
                 'Vehicle Count']]

In [10]:
# export data to csv
df4

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count
0,MAERSK BLR,MAERSK BLR,TRIP,BLR,2024-08-06,1,0,0,0,ICE,12.0
1,CALIXINDIABLR,CALIXINDIA-RMZ INFINITY-BANGALORE,TRIP,BLR,2024-07-18,34,0,0,0,ICE,4.0
2,MAERSK BLR,MAERSK BLR,TRIP,BLR,2024-07-10,2,0,0,0,ICE,10.0
3,FRACTALMUM,FRACTALMUM-MUM-FRACTALMUM,DEDICATED_12,MUM,2024-07-18,2,0,0,0,ICE,4.0
4,SAP,SAP-BLR-SAP LABS EPIP,TRIP,BLR,2024-08-06,3,0,0,0,ICE,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5589,Bangalore-Spot,BANGALORE-SPOT,SPOT,BLR,2024-07-31,15,0,MG ZS EV,0,Spot,NaN
5590,Mumbai-Spot,MUMBAI-SPOT,SPOT,MUM,2024-07-08,1,0,E6,0,Spot,14.0
5591,Bangalore-Spot,BANGALORE-SPOT,SPOT,BLR,2024-07-18,4,0,E6,0,Spot,NaN
5592,Mumbai-Spot,MUMBAI-SPOT,SPOT,MUM,2024-07-03,2,0,E6,0,Spot,12.0


In [11]:
# Install the gspread library
!pip install gspread

In [12]:
# Import necessary libraries
from google.colab import auth
import gspread
from google.auth import default # Import the default method
import pandas as pd

# Authenticate with Google
auth.authenticate_user()

# Get default credentials
creds, _ = default() # Obtain credentials using the default method

# Authorize gspread using the obtained credentials
gc = gspread.authorize(creds)

# Open the Google Sheet
sheet = gc.open('Client rate master').sheet1
# open second sheet also
sheet2 = gc.open('Client rate master').get_worksheet(1)
sheet3 = gc.open('Client rate master').get_worksheet(2)
# Get all values from the sheet
data1 = sheet.get_all_values()
data3= sheet3.get_all_values()
data2= sheet2.get_all_values()
# Convert to a Pandas DataFrame
df5 = pd.DataFrame(data1[1:], columns=data1[0])
df6 = pd.DataFrame(data2[1:], columns=data2[0])
df7 = pd.DataFrame(data3[1:], columns=data3[0])

In [13]:
df6

,Sr No,Site Name,Model,Packaging hour,billing model,Rate
0,1,BANGALORE-SPOT,E-Verito,0,Spot,1400
1,2,BANGALORE-SPOT,TIGOR EV,0,Spot,1400
2,3,BANGALORE-SPOT,ZIPTRON,0,Spot,1400
3,4,BANGALORE-SPOT,MG ZS EV,0,Spot,2000
4,5,BANGALORE-SPOT,E6,0,Spot,2400
...,...,...,...,...,...,...
275,276,KOLKATA-SPOT,BYD E6,10,Spot,3000
276,277,MUMBAI-SPOT,BYD E6,10,Spot,3000
277,278,PUNE-SPOT,BYD E6,10,Spot,3000
278,279,CHENNAI-SPOT,BYD E6,10,Spot,3000


In [14]:
# take all these column from df5 'Client Office','Client','Billing Model','Rate','Escort Rate'
df5=df5[['Client Office','Client','Billing Model','Rate','Escort Rate']]

In [15]:
# make upper case Client Office
df5['Client Office']=df5['Client Office'].str.upper()
df5['Billing Model']=df5['Billing Model'].str.upper()

In [16]:
df5

,Client Office,Client,Billing Model,Rate,Escort Rate
0,HULBELGAUM-BGM-HULBELGAUM,HUL,DEDICATED_12,"65,000",0
1,24BY7BLR-BLR-24BY7 PRESTIGE TECH PARK,24*7 AI,DEDICATED_24,"94,000",0
2,ACCENT-BLR-B10_BAGMANE TECH PARK,Accenture,DEDICATED_24,92500,0
3,ACCENT-BLR-B9_PRESTIGE TECH PARK DAIRY CIRCLE,Accenture,DEDICATED_24,92500,0
4,ACCENT-BLR-BDC7 PRITECH PARK SEZ-D24,Accenture,DEDICATED_24,"92,500",0
...,...,...,...,...,...
169,BIRLASOFTPNQ-PNQ-BIRLASOFT PUNE,BIRLASOFT,TRIP,700,0
170,HAPAG_LLOYD_ITHINK_TOWER-MUMBAI,HAPAG_LLOYD,DEDICATED_12,76000,0
171,KPMG-ADVANT NAVIS BUSINESS PARK-NOIDA,KPMG,DEDICATED_24,79000,0
172,TDCXHYD-HYD-TDCXHYD,TDX,DEDICATED_24,113000,0


In [17]:
import pandas as pd

# Merge the dataframes
merged_df = pd.merge(df5, df4,
                     left_on=['Client Office', 'Billing Model'],
                     right_on=['client_office', 'billing_model'],
                     how='outer')

# Ensure 'Rate' from df5 is renamed to 'Rates'
merged_df = merged_df.rename(columns={'Rate': 'Rates'})

# Select the desired columns from the merged dataframe
df4 = merged_df[['client','client_office', 'billing_model', 'city_code',
                 'trip_date', 'trip_count',
                 'escort_count', 'model', 'package_hour', 'cet_models',
                 'Vehicle Count', 'Rates','Escort Rate']]


In [18]:
df4

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-01,2.0,0.0,0,0.0,CET,1.0,"65,000",0
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-02,2.0,0.0,0,0.0,CET,1.0,"65,000",0
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-03,2.0,0.0,0,0.0,CET,1.0,"65,000",0
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-04,2.0,0.0,0,0.0,CET,1.0,"65,000",0
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-08,2.0,0.0,0,0.0,CET,1.0,"65,000",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5702,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-29,8.0,0.0,Ziptron,0.0,Spot,NaN,NaN,NaN
5703,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-14,1.0,0.0,Ziptron,0.0,Spot,NaN,NaN,NaN
5704,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-20,1.0,0.0,Ziptron,0.0,Spot,NaN,NaN,NaN
5705,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-12,2.0,0.0,Ziptron,0.0,Spot,NaN,NaN,NaN


In [19]:
df6['billing model']=df6['billing model'].str.upper()
df4['model']=df4['model'].str.upper()
df4['billing_model']=df4['billing_model'].str.upper()

<ipython-input-19-6be71923fb94>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['model']=df4['model'].str.upper()
<ipython-input-19-6be71923fb94>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['billing_model']=df4['billing_model'].str.upper()


In [20]:
# seperate df4 data where billing_model='DEDICATED_12',DEDICATED_24,TRIP AND CREAT SEPERATE DF FOR SPOT
df_DEDICATED = df4[df4['billing_model'].isin(['DEDICATED_12', 'DEDICATED_24', 'TRIP'])]
DF_SPOT = df4[df4['billing_model'] == 'SPOT']

In [21]:
df_DEDICATED

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-01,2.0,0.0,0,0.0,CET,1.0,"65,000",0
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-02,2.0,0.0,0,0.0,CET,1.0,"65,000",0
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-03,2.0,0.0,0,0.0,CET,1.0,"65,000",0
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-04,2.0,0.0,0,0.0,CET,1.0,"65,000",0
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-08,2.0,0.0,0,0.0,CET,1.0,"65,000",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5169,VODAFONEPNQ,PDC3-SP INFOCITY - VODAFONE-PNQ,TRIP,PNQ,2024-08-01,1.0,0.0,0,0.0,CET,NaN,NaN,NaN
5170,VODAFONEPNQ,PDC3-SP INFOCITY - VODAFONE-PNQ,TRIP,PNQ,2024-08-06,11.0,0.0,0,0.0,CET,NaN,NaN,NaN
5171,VODAFONEPNQ,PDC3-SP INFOCITY - VODAFONE-PNQ,TRIP,PNQ,2024-08-07,28.0,2.0,0,0.0,CET,NaN,NaN,NaN
5172,VODAFONEPNQ,PDC3-SP INFOCITY - VODAFONE-PNQ,TRIP,PNQ,2024-08-08,24.0,1.0,0,0.0,CET,NaN,NaN,NaN


In [22]:
DF_SPOT

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate
5174,Mumbai-Spot,MUMBAI-SPOT,SPOT,MUM,2024-07-10,3.0,0.0,MG ZS EV,8.0,Spot,14.0,NaN,NaN
5175,Mumbai-Spot,MUMBAI-SPOT,SPOT,MUM,2024-07-16,2.0,0.0,E6,0.0,Spot,13.0,NaN,NaN
5176,Mumbai-Spot,MUMBAI-SPOT,SPOT,MUM,2024-07-09,1.0,0.0,ZIPTRON,8.0,Spot,14.0,NaN,NaN
5177,Mumbai-Spot,MUMBAI-SPOT,SPOT,MUM,2024-08-02,3.0,0.0,ZIPTRON,0.0,Spot,12.0,NaN,NaN
5178,Mumbai-Spot,MUMBAI-SPOT,SPOT,MUM,2024-07-29,1.0,0.0,ZIPTRON,0.0,Spot,12.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5702,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-29,8.0,0.0,ZIPTRON,0.0,Spot,NaN,NaN,NaN
5703,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-14,1.0,0.0,ZIPTRON,0.0,Spot,NaN,NaN,NaN
5704,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-20,1.0,0.0,ZIPTRON,0.0,Spot,NaN,NaN,NaN
5705,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-12,2.0,0.0,ZIPTRON,0.0,Spot,NaN,NaN,NaN


In [23]:
# DF_SPOT package_hour convert into int64
DF_SPOT['package_hour'] = DF_SPOT['package_hour'].astype('int64')
df6['Packaging hour'] = df6['Packaging hour'].astype('int64')

<ipython-input-23-611d883efcc2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_SPOT['package_hour'] = DF_SPOT['package_hour'].astype('int64')


In [24]:
print(DF_SPOT.columns)
print(df6.columns)

Index(['client', 'client_office', 'billing_model', 'city_code', 'trip_date',
       'trip_count', 'escort_count', 'model', 'package_hour', 'cet_models',
       'Vehicle Count', 'Rates', 'Escort Rate'],
      dtype='object')
Index(['Sr No ', 'Site Name ', 'Model', 'Packaging hour', 'billing model',
       'Rate'],
      dtype='object')


In [25]:
# Merge the dataframes
merged_df = pd.merge(DF_SPOT,df6,
                     left_on=['client_office','package_hour','model'],
                     right_on=['Site Name ','Packaging hour','Model'],
                     how='outer')

# Ensure 'Rate' from df5 is renamed to 'Rates'
merged_df = merged_df.rename(columns={'Rate': 'spotRates'})

# Select the desired columns from the merged dataframe
df4 = merged_df[['client','client_office', 'billing_model', 'city_code',
                 'trip_date', 'trip_count',
                 'escort_count', 'model', 'package_hour', 'cet_models',
                 'Vehicle Count', 'spotRates','Escort Rate']]


In [26]:
# show f4 data where client_office is not null
df4=df4[df4['client_office'].notnull()]

In [27]:
df4

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,spotRates,Escort Rate
0,Mumbai-Spot,MUMBAI-SPOT,SPOT,MUM,2024-07-10,3.0,0.0,MG ZS EV,8.0,Spot,14.0,3400,NaN
1,Mumbai-Spot,MUMBAI-SPOT,SPOT,MUM,2024-08-02,1.0,0.0,MG ZS EV,8.0,Spot,12.0,3400,NaN
2,Mumbai-Spot,MUMBAI-SPOT,SPOT,MUM,2024-07-30,1.0,0.0,MG ZS EV,8.0,Spot,12.0,3400,NaN
3,Mumbai-Spot,MUMBAI-SPOT,SPOT,MUM,2024-07-03,2.0,0.0,MG ZS EV,8.0,Spot,12.0,3400,NaN
4,Mumbai-Spot,MUMBAI-SPOT,SPOT,MUM,2024-07-26,1.0,0.0,MG ZS EV,8.0,Spot,13.0,3400,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-29,8.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN
529,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-14,1.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN
530,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-20,1.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN
531,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-12,2.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN


In [28]:
# Combine DataFrames vertically
combined_df = pd.concat([df_DEDICATED
,df4], axis=0, ignore_index=True)

In [29]:
combined_df

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate,spotRates
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-01,2.0,0.0,0,0.0,CET,1.0,"65,000",0,NaN
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-02,2.0,0.0,0,0.0,CET,1.0,"65,000",0,NaN
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-03,2.0,0.0,0,0.0,CET,1.0,"65,000",0,NaN
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-04,2.0,0.0,0,0.0,CET,1.0,"65,000",0,NaN
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-08,2.0,0.0,0,0.0,CET,1.0,"65,000",0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5656,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-29,8.0,0.0,ZIPTRON,0.0,Spot,NaN,NaN,NaN,1400
5657,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-14,1.0,0.0,ZIPTRON,0.0,Spot,NaN,NaN,NaN,1400
5658,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-20,1.0,0.0,ZIPTRON,0.0,Spot,NaN,NaN,NaN,1400
5659,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-12,2.0,0.0,ZIPTRON,0.0,Spot,NaN,NaN,NaN,1400


In [30]:
import numpy as np

In [31]:
# Update the Rate column conditionally
combined_df['Rates'] = np.where(
    (combined_df['billing_model'] == 'SPOT') & (combined_df['Rates'].isna()),
    combined_df['spotRates'],
    combined_df['Rates']
)

print("\nUpdated DataFrame df1:")
print(combined_df)



Updated DataFrame df1:
            client              client_office billing_model city_code  \
0       HULBELGAUM  HULBELGAUM-BGM-HULBELGAUM  DEDICATED_12       BGM   
1       HULBELGAUM  HULBELGAUM-BGM-HULBELGAUM  DEDICATED_12       BGM   
2       HULBELGAUM  HULBELGAUM-BGM-HULBELGAUM  DEDICATED_12       BGM   
3       HULBELGAUM  HULBELGAUM-BGM-HULBELGAUM  DEDICATED_12       BGM   
4       HULBELGAUM  HULBELGAUM-BGM-HULBELGAUM  DEDICATED_12       BGM   
...            ...                        ...           ...       ...   
5656  Chennai-Spot               CHENNAI-SPOT          SPOT       CHN   
5657  Chennai-Spot               CHENNAI-SPOT          SPOT       CHN   
5658  Chennai-Spot               CHENNAI-SPOT          SPOT       CHN   
5659  Chennai-Spot               CHENNAI-SPOT          SPOT       CHN   
5660  Chennai-Spot               CHENNAI-SPOT          SPOT       CHN   

       trip_date  trip_count  escort_count    model  package_hour cet_models  \
0     2024-07-01   

In [32]:
combined_df

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate,spotRates
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-01,2.0,0.0,0,0.0,CET,1.0,"65,000",0,NaN
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-02,2.0,0.0,0,0.0,CET,1.0,"65,000",0,NaN
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-03,2.0,0.0,0,0.0,CET,1.0,"65,000",0,NaN
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-04,2.0,0.0,0,0.0,CET,1.0,"65,000",0,NaN
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-08,2.0,0.0,0,0.0,CET,1.0,"65,000",0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5656,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-29,8.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN,1400
5657,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-14,1.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN,1400
5658,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-20,1.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN,1400
5659,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-12,2.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN,1400


In [33]:
# drop column spotRates from combined_df
combined_df=combined_df.drop(columns=['spotRates'])

In [34]:
combined_df

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-01,2.0,0.0,0,0.0,CET,1.0,"65,000",0
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-02,2.0,0.0,0,0.0,CET,1.0,"65,000",0
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-03,2.0,0.0,0,0.0,CET,1.0,"65,000",0
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-04,2.0,0.0,0,0.0,CET,1.0,"65,000",0
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-08,2.0,0.0,0,0.0,CET,1.0,"65,000",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5656,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-29,8.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN
5657,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-14,1.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN
5658,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-20,1.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN
5659,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-12,2.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN


In [35]:
client_office_22 =  [
    "HULBELGAUM-BGM-HULBELGAUM",
    "AdobeB-BLR-Prestige Tech Park",
    "AONB-BLR-AON",
    "ASCENDUM-BLR-ASCENDUM",
    "BNPBLR-BLR-BNPP_BAGMANE",
    "Capco-BLR-Capco SJP",
    "COLTTE-BLR-MTP",
    "DAIMLERB-BLR-DAIMLER",
    "EverestBLR-BLR-Manyata Tech Park_D12_MG",
    "EverestBLR-BLR-Manyata Tech Park_D12_Sedan",
    "Fractal-BLR-FractalVTP",
    "GOOGLEBLR-BLR-GOOGLE BCPC",
    "HULBAN-BLR-PRESTIGE SHANTI NIKETAN",
    "KPMG-BLR-ECOWORLD",
    "KPMG-BLR-ECOWORLD",
    "Maersk Blr",
    "MERCED-BLR-EMBASSY CREST",
    "OPTUM-BLR-OPTUM JP NAGAR",
    "OPTUM-BLR-OPTUM_ADUGODI",
    "OPTUM-BLR-OPTUM_BANASWADI",
    "PaloAltoNetworksBLR-BLR-Palo Alto Networks",
    "SALESF-BLR-EGL",
    "SALESFORCE-BLR-CBP",
    "SALESFORCE-BLR-CBP",
    "SAP-BLR-SAP ARIBA",
    "SAP-BLR-SAP LABS EPIP",
    "SASKEN-BLR-EGL",
    "SHELLBLR-BLR-SHELL_BLR",
    "TelstraBLR-BLR-TelstraBLR",
    "WELLSFARGOBLR-BLR-EMBASSY TECH PARK-D12",
    "WellsFargoBLR-BLR-Embassy Tech Park-D12",
    "WiproB-BLR-Google_Youtube Bagmane Tech park-Trip",
    "WiproB-BLR-Kodathi",
    "WiproB-BLR-Sarjapur",
    "TitanCOIMB-Coimbatore-Sulur TN_MG",
    "ProdaptCHN-CHN-SIDCO Ind. Estate Guindy_D24",
    "ShellCHN-CHN-Shell_CHN",
    "BIRLASOFTHYD-HYD-BIRLASOFT",
    "CORTEVAHYD-HYD-WAVEROCK-TRIP",
    "COTTELIGENTHYD-HYD-COTTELIGENT",
    "DUPONTHYD-HYD-DUPONT_WAVEROCK",
    "GOOGLEHYD-HYD-OMEGA",
    "GOOGLEHYD-HYD-OMEGA",
    "GOOGLEHYD-HYD-SALARPURIYA SATTVA",
    "JPMCHy-HYD-JPMC Tower",
    "Novart-HYD-Novartis",
    "PEPSICOHYD-HYD-PEPSICO",
    "VirtusaHyd-HYD-VirtusaHyd",
    "VIRTUSAHYD-HYD-VIRTUSAHYD",
    "WELLSFARGOHYD-HYD-HITECH CITY-D12",
    "ICICIIDR-IDR-ICICIIDR",
    "GenpactKOL-KOL-Candor Tech Space-D12",
    "BNPMUM-MUM-BNP_INFINITY_IT_PARK",
    "FRACTALMUM-MUM-FRACTALMUM",
    "GenpactMUM-MUM-Raiaskaran Tech Park-D12",
    "MICMUM-MUM-BKC",
    "THELALITMUM-MUM-SAHAR AIRPORT ROAD_D24",
    "AONNCR-NCR-AON_CANDOR_G1",
    "Blackrock-DLF Cyber City 14B-Gurgaon",
    "COLT-NCR-CANDOR",
    "FRACTALNCR-NCR-FRACTALNCR",
    "GoogleNCR-NCR-Google",
    "GOOGLENCR-NCR-GOOGLE",
    "Incedo-NCR-DLF Cyber Park-Trip",
    "KPMGNCR-NCR-KPMG_DLF_CYBERCITY",
    "RBSNCR-NCR-RBSNCR",
    "SAPNCR-NCR-SAP",
    "SAPNCR-NCR-SAP-SDC",
    "WIPRON-NCR-CANDOR-TRIP",
    "WiproN-NCR-Wipro-ONYX",
    "ADPPUN-PUN-VIMANNAGAR-TRIP",
    "BIRLASOFTPNQ-PNQ-BIRLASOFT PUNE",
    "BNYMellonPNQ-PNQ-BNYMellon",
    "CAPCOPNQ-PNQ-VATIKA BUSINESS PARK",
    "CapPun-PNQ-Capgemini Hinjewadi Phase 3",
    "CREDIT-PNQ-EON_IT_PARK",
    "FISERV-PNQ-FISERV",
    "GenpactPNQ-PNQ-Nyati Tech Park-D12",
    "INFOSYSPNQ-PNQ-INFOSYS-PUNE",
    "SAPPNQ-PNQ-SAP PUNE",
    "TELSTRAPUNE-PUNE-QBP",
    "WellsFargoBLR-BLR-Embassy Tech Park",
    "WellsFargoBLR-BLR-Embassy Tech Park",
    "COMPUGAIN SOLUTIONS-WAVEROCK-HYDERABAD",
    "WELLSFARGO-EMBASSY SPLENDID TECH ZONE-CHENNAI",
    "WELLSFARGO-EMBASSY SPLENDID TECH ZONE-CHENNAI",
    "WELLSFARGOHYD-HYD-HITECH CITY-D12",
    "MCKINSEYGLOBAL-WORLD TRADE CENTER PERUNGUDI-CHENNAI",
    "ICICIBANK-NEAR SARITAVIHAR METRO STATION-NEW DELHI",
    "AON-LOTUS BUSINESS PARK-NOIDA",
    "PRINCIPALGLOBAL-TOWER 16 CYBERCITY-PUNE","EVERESTBLR-BLR-MANYATA TECH PARK_D12_MG",
"FRACTAL-BLR-FRACTALVTP","MAERSK BLR","PALOALTONETWORKSBLR-BLR-PALO ALTO NETWORKS","TELSTRABLR-BLR-TELSTRABLR","SHELLCHN-CHN-SHELL_CHN","NOVART-HYD-NOVARTIS","INCEDO-NCR-DLF CYBER PARK-TRIP","WIPRON-NCR-WIPRO-ONYX","WELLSFARGOBLR-BLR-EMBASSY TECH PARK","KPMG-ADVANT NAVIS BUSINESS PARK-NOIDA","HAPAG_LLOYD_ITHINK_TOWER-MUMBAI"]
client_office_26 = ["24BY7BLR-BLR-24BY7 PRESTIGE TECH PARK",
    "ACCENT-BLR-B10_BAGMANE TECH PARK",
    "ACCENT-BLR-B9_PRESTIGE TECH PARK DAIRY CIRCLE",
    "ACCENT-BLR-BDC7 PRITECH PARK SEZ-D24",
    "ADOBEB-BLR-PRESTIGE TECH PARK",
    "ANZBLR-BLR-ANZ_MANYATHA TECH PARK",
    "HGSBLR-BLR-HGSBLR",
    "SAMSUN-BLR-BAGMANE CONSTELLATION",
    "SONICWALL-BLR-EMBASSY TECH-TRIP",
    "TescoBlr-BLR-Tescocampus",
    "UNIBLR-BLR-UNISYS RGA",
    "VMWARE-BLR-KALYANI VISTA",
    "VolvoIndia-BLR-Volvo Bangalore",
    "TitanCOIMB-Coimbatore-Sulur TN_Sedan",
    "MERCURE-CHN-SRIPERUMBUDUR-BYD",
    "MERCURE-CHN-SRIPERUMBUDUR-Sedan",
    "TITANHSR-HSR-TITAN_HOSUR",
    "AccentureHYD-HYD-Diyashree-Orion-D24",
    "AccentureHYD-HYD-Diyashree-Orion-D24 + ACCENTUREHYD-HYD-MANTRI CELESTIA-D24",
    "ACCENTUREHYD-HYD-MANTRI CELESTIA-D24",
    "ADPHYD-HYD-NANAKARAMGUDA-TRIP",
    "CYBERCITYHYD-HYD-DLFCYBERCITYHYD",
    "GenpactHYD-HYD-Genpact_HYD",
    "ICICIHYD-HYD-ICICIHYD",
    "SAGILITY-HYD-PURVA SUMMIT HITEC CITY-D24",
    "TDCXHYD-HYD-TDCXHYD",
    "Unisys-DLF Cybercity-Hyderabad",
    "WIPROH-HYD-UBS-LIBRA-EBLOCK-TRIP",
    "WIPROH-HYD-WIPRO MANDICONDA",
    "MANIPA-MPL-MAHE MANIPAL",
    "ACCENTUREMUM-MUM-GODREJ IT PARK-D24",
    "JPMCMu-MUM-Prism",
    "ACCENTURENCR-NCR-UNITECH INFOSPACE-D24",
    "ADOBEN-NCR-ADOBE",
    "GenpactNCR-NCR-GenpactNCR",
    "UNISYSNCR-NCR-DLF CYBER GREEN GURUGRAM-TRIP",
    "ZSNCR-NCR-WORLD TECH PARK-D24",
    "AXAPNQ-PNQ-AXA BSS Pune",
    "BARPUN-PNQ-GERA COMMERCE ZONE",
    "ECLERXSERVICESLTD-PNQ-HINJEWADI",
    "MICPUN-PNQ-DIGHI",
    "MICPUN-PNQ-HINJEWADI",
    "VodafonePNQ-PNQ-Vodafone EON",
    "VODAFONEPNQ-PNQ-VODAFONE_MANTRI",
    "ZSPUNE-PNQ-ZS PUNE",
    "VolvoIndia-BLR-Volvo Bangalore",
    "BoschGlobal-Adugodi-Bangalore",
    "IHS Global-Hoodi Industrial Area-Bangalore",
    "INNOMINDS-WAVEROCK NANAKRAMGUDA VILLAGE-HYDERABAD",
    "S&P CAPITAL-SKYVIEW 10-HYDERABAD",
    "WIPRO-GOPANPALLY-HYDERABAD",
    "LBADDINCR-NCR-LOREAL BADDI",
    "S&P CAPITAL-DLF CYBERCITY-GURGAON",
    "WIPRO-GMR NEW UDAAN BHAWAN-NCR",
    "WIPRO-IAF SUBROTO PARK-GURGAON",
    "MARKIT INDIA-AMBIENCE CORPORATE TOWER II-GURGAON",
    "MARKIT INDIA-CANDOR TECHSPACE-NOIDA",
    "Amdocs-PNQ-Amdocs_Magarpatta",
    "KRONOSSOLUTIONS-OKAYA TOWERS-NOIDA",
    "Barclay-DLF IT Park Manapakkam-Chennai",
    "AMAZONDATA-HITECH CITY MADHAPUR-HYDERABAD",
    "AMAZONDATA-NANAKRAMGUDA-HYDERABAD",
    "TITANCOIMB-COIMBATORE-SULUR TN_D12",
    "SYNCHRONY-SALARPURIYA SATTVA KNOWLEDGE PARK-HYDERABAD",
    "MERCURE-CHN-SRIPERUMBUDUR-D24",
    "BIRLASOFT-MIDC MAHAPE-NAVI MUMBAI",
    "HCL-JIGANI SPECIAL ECONOMIC ZONE-BLR",
    "ACCENTURE-GIGAPLEX IT PART AIROLI-MUMBAI",
    "UNISYS-WAVEROCK-HYDERABAD","TESCOBLR-BLR-TESCOCAMPUS"
,"VOLVOINDIA-BLR-VOLVO BANGALORE"
,"GENPACTHYD-HYD-GENPACT_HYD",
"UNISYS-DLF CYBERCITY-HYDERABAD",
"JPMCMU-MUM-PRISM",
"BARNCR-NCR-BARCLAYS CANDOR N1"
,"BLACKROCK-DLF CYBER CITY 14B-GURGAON"
,"GENPACTNCR-NCR-GENPACTNCR",
"AXAPNQ-PNQ-AXA BSS PUNE"
,"VODAFONEPNQ-PNQ-VODAFONE EON",
"BOSCHGLOBAL-ADUGODI-BANGALORE",
"IHS GLOBAL-HOODI INDUSTRIAL AREA-BANGALORE"
,"BARCLAY-DLF IT PARK MANAPAKKAM-CHENNAI","VIRTUSA ECO WORLD RMZ BLR","PDC3-SP INFOCITY - VODAFONE-PNQ","MERCURE-CHN-SRIPERUMBUDUR","PRODAPTCHN-CHN-SIDCO IND. ESTATE GUINDY"
]
client_office_31=["TheLalitNCR-NCR-Firebrigadelane_D24","EPICORSOFTWARE-RASHINI TECH HUB-BANGALORE","UNIQUEMERCANTILE-NAGAVARA-BANGALORE",
                  "STAYPRIMROSE- MUSSOORIE-DEHRADUN","CALIXINDIA-RMZ INFINITY-BANGALORE","THELALITNCR-NCR-FIREBRIGADELANE_D24","BNYMELLONPNQ-PNQ-BNYMELLON"
]

In [36]:
from datetime import datetime, timedelta


*FUnction 1


In [37]:
# Define the main function to set the day count dynamically
def set_dynamic_day_count(client_office, year, month):
    start_date = datetime(year, month, 1)

    # Determine the current month's total days
    current_month_days = working_days_excluding(start_date, 31, exclude_saturdays=True, exclude_sundays_only=False)

    if client_office in client_office_22:
        # Exclude both Saturdays and Sundays for 22 days client office
        working_days = working_days_excluding(start_date, 22, exclude_saturdays=True, exclude_sundays_only=True)
    elif client_office in client_office_26:
        # Exclude only Sundays for 26 days client office
        working_days = working_days_excluding(start_date, 26, exclude_saturdays=False, exclude_sundays_only=True)
    else:
        # Default is 31 days with no exclusions
        working_days = current_month_days

    return working_days

# Calculating Working Days #


In [38]:
# Define function to calculate working days excluding specific days
def working_days_excluding(start_date, num_days, exclude_saturdays=True, exclude_sundays_only=False):
    current_date = start_date
    working_days_count = 0
    total_days = 0
    while total_days < 31:  # Consider up to 31 days to handle all possible days in a month
        if exclude_saturdays and current_date.weekday() == 5:  # Saturday
            pass
        elif exclude_sundays_only and current_date.weekday() == 6:  # Sunday
            pass
        elif exclude_saturdays and current_date.weekday() == 6:  # Sunday
            pass
        else:
            working_days_count += 1
        current_date += timedelta(days=1)
        total_days += 1
        if current_date.month != start_date.month:  # End of month
            break
    return working_days_count


#Main Code for Dynamic Day Count#

In [39]:
# Define the main function to set the day count dynamically
def set_dynamic_day_count(client_office, year, month):
    start_date = datetime(year, month, 1)

    # Determine the current month's total days
    # Set to 31 as maximum days to handle month-end cases
    max_days = 31
    current_month_days = working_days_excluding(start_date, max_days, exclude_saturdays=False, exclude_sundays_only=False)

    if client_office in client_office_22:
        # Exclude both Saturdays and Sundays for 22 days client office
        working_days = working_days_excluding(start_date, 22, exclude_saturdays=True, exclude_sundays_only=True)
    elif client_office in client_office_26:
        # Exclude only Sundays for 26 days client office
        working_days = working_days_excluding(start_date, 26, exclude_saturdays=False, exclude_sundays_only=True)
    elif client_office in client_office_31:
        # Default to 31 days with no exclusions for 31 days client office
        working_days = working_days_excluding(start_date, max_days, exclude_saturdays=False, exclude_sundays_only=False)
    else:
        # Handle unknown client office cases if needed
        working_days = None

    return working_days

# Previous month and current month FUnction #

In [40]:
# Define function to calculate both current and previous month day counts based on trip_date
def get_monthly_day_counts(client_office, trip_date):
    year = trip_date.year
    current_month = trip_date.month

    # Determine the previous month
    if current_month == 1:
        previous_month = 12
        previous_year = year - 1
    else:
        previous_month = current_month - 1
        previous_year = year

    # Get day counts for the current and previous month
    current_month_day_count = set_dynamic_day_count(client_office, year, current_month)
    previous_month_day_count = set_dynamic_day_count(client_office, previous_year, previous_month)

    return current_month_day_count, previous_month_day_count

In [41]:
# Remove commas and spaces, then convert to numeric (int64)
combined_df['Rates'] = pd.to_numeric(combined_df['Rates'].str.replace(',', '').str.replace(' ', ''), errors='coerce').fillna(0).astype('int64')

In [42]:
combined_df.columns

Index(['client', 'client_office', 'billing_model', 'city_code', 'trip_date',
       'trip_count', 'escort_count', 'model', 'package_hour', 'cet_models',
       'Vehicle Count', 'Rates', 'Escort Rate'],
      dtype='object')

In [43]:
# Define function to calculate per-day rate based on billing model
def calculate_per_day_rate(billing_model, Rates, current_month_day_count):
    if billing_model in ["DEDICATED_12", "DEDICATED_24"]:
        return Rates / current_month_day_count
    elif billing_model in ["TRIP", "SPOT"]:
        return Rates



# Conver Trip Date to timedate#

In [44]:
# Convert 'tripdate' column to datetime
combined_df['trip_date'] = pd.to_datetime(combined_df['trip_date'])

#Apply to COmbined Df #

In [45]:
# Define function to process the DataFrame
def process_combined_df(combined_df):


    # Apply the function to update the combined_df with the dynamic day count
    combined_df[['current_month_day_count', 'previous_month_day_count']] = combined_df.apply(
        lambda row: pd.Series(get_monthly_day_counts(row['client_office'], row['trip_date'])), axis=1
    )

    # Calculate per_day_rate based on billing_model
    combined_df['per_day_rate'] = combined_df.apply(
        lambda row: round(calculate_per_day_rate(row['billing_model'], row['Rates'], row['current_month_day_count']),0),
        axis=1
    )

    return combined_df

In [46]:
combined_df.head()

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-01,2.0,0.0,0,0.0,CET,1.0,65000,0
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-02,2.0,0.0,0,0.0,CET,1.0,65000,0
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-03,2.0,0.0,0,0.0,CET,1.0,65000,0
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-04,2.0,0.0,0,0.0,CET,1.0,65000,0
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-08,2.0,0.0,0,0.0,CET,1.0,65000,0


In [47]:
# Process the DataFrame
processed_df = process_combined_df(combined_df)

In [48]:
processed_df.head()


,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate,current_month_day_count,previous_month_day_count,per_day_rate
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-01,2.0,0.0,0,0.0,CET,1.0,65000,0,23.0,20.0,2826.0
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-02,2.0,0.0,0,0.0,CET,1.0,65000,0,23.0,20.0,2826.0
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-03,2.0,0.0,0,0.0,CET,1.0,65000,0,23.0,20.0,2826.0
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-04,2.0,0.0,0,0.0,CET,1.0,65000,0,23.0,20.0,2826.0
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-08,2.0,0.0,0,0.0,CET,1.0,65000,0,23.0,20.0,2826.0


In [49]:
combined_df.drop(columns=["previous_month_day_count"],inplace=True)

In [50]:
# export csv
combined_df

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate,current_month_day_count,per_day_rate
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-01,2.0,0.0,0,0.0,CET,1.0,65000,0,23.0,2826.0
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-02,2.0,0.0,0,0.0,CET,1.0,65000,0,23.0,2826.0
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-03,2.0,0.0,0,0.0,CET,1.0,65000,0,23.0,2826.0
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-04,2.0,0.0,0,0.0,CET,1.0,65000,0,23.0,2826.0
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-08,2.0,0.0,0,0.0,CET,1.0,65000,0,23.0,2826.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5656,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-29,8.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN,NaN,1400.0
5657,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-14,1.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN,NaN,1400.0
5658,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-20,1.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN,NaN,1400.0
5659,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-12,2.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN,NaN,1400.0


In [51]:
combined_df

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate,current_month_day_count,per_day_rate
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-01,2.0,0.0,0,0.0,CET,1.0,65000,0,23.0,2826.0
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-02,2.0,0.0,0,0.0,CET,1.0,65000,0,23.0,2826.0
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-03,2.0,0.0,0,0.0,CET,1.0,65000,0,23.0,2826.0
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-04,2.0,0.0,0,0.0,CET,1.0,65000,0,23.0,2826.0
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-08,2.0,0.0,0,0.0,CET,1.0,65000,0,23.0,2826.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5656,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-29,8.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN,NaN,1400.0
5657,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-14,1.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN,NaN,1400.0
5658,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-20,1.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN,NaN,1400.0
5659,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-12,2.0,0.0,ZIPTRON,0.0,Spot,NaN,1400,NaN,NaN,1400.0


In [52]:
df7

,City,Client Name,Client_office,Sales_force,Billing_Model,Normative_EV_ount_July
0,BLR,BNP(Moveinsync),BNPBLR-BLR-BNPP_BAGMANE,Bnp-Bagmane-Bangalore,DEDICATED_12,5
1,BLR,DAIMLER,DAIMLERB-BLR-DAIMLER,Diamler- Whitefield-Bangalore,DEDICATED_12,2
2,BLR,GOOGLE,GOOGLEBLR-BLR-GOOGLE BCPC,Google-Bcpc-Bangalore,DEDICATED_12,88
3,BLR,HUL,HULBAN-BLR-PRESTIGE SHANTI NIKETAN,Hindustan Unilever-Shantiniketan-Bangalore,DEDICATED_12,1
4,BLR,KPMG,KPMG-BLR-ECOWORLD,Kpmg-Ecoworld-Bangalore,DEDICATED_12,12
...,...,...,...,...,...,...
175,CHN,,MERCURE-CHN-SRIPERUMBUDUR,,DEDICATED_24,5
176,CHN,,PRODAPTCHN-CHN-SIDCO IND. ESTATE GUINDY,,DEDICATED_24,1
177,HYD,,TDCXHYD-HYD-TDCXHYD,,DEDICATED_24,
178,HYD,,VIRTUSAHYD-HYD-VIRTUSAHYD,,DEDICATED_24,


In [53]:
combined_df.columns

Index(['client', 'client_office', 'billing_model', 'city_code', 'trip_date',
       'trip_count', 'escort_count', 'model', 'package_hour', 'cet_models',
       'Vehicle Count', 'Rates', 'Escort Rate', 'current_month_day_count',
       'per_day_rate'],
      dtype='object')

In [54]:
df7.columns

Index(['City', 'Client Name', 'Client_office', 'Sales_force', 'Billing_Model',
       'Normative_EV_ount_July'],
      dtype='object')

In [55]:
# Merge the dataframes
merged_df = pd.merge(combined_df,df7,
                     left_on=['client_office','billing_model'],
                     right_on=['Client_office','Billing_Model'],
                     how='outer')

# Ensure 'Rate' from df5 is renamed to 'Rates'
merged_df = merged_df.rename(columns={'Normative_EV_ount_July': 'Plan Vehicle/Trip'})

# Select the desired columns from the merged dataframe
df4 = merged_df[['client','client_office', 'billing_model', 'city_code', 'trip_date',
       'trip_count', 'escort_count', 'model', 'package_hour', 'cet_models',
       'Vehicle Count', 'Rates', 'Escort Rate', 'current_month_day_count',
       'per_day_rate','Plan Vehicle/Trip']]


In [56]:
df4

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate,current_month_day_count,per_day_rate,Plan Vehicle/Trip
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-01,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-02,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-03,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-04,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-08,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5955,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20
5956,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
5957,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
5958,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [57]:
# show f4 data where client_office is not null
df4=df4[df4['client_office'].notnull()]

In [58]:
df4

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate,current_month_day_count,per_day_rate,Plan Vehicle/Trip
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-01,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-02,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-03,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-04,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-08,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5915,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-29,8.0,0.0,ZIPTRON,0.0,Spot,NaN,1400.0,NaN,NaN,1400.0,NaN
5916,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-14,1.0,0.0,ZIPTRON,0.0,Spot,NaN,1400.0,NaN,NaN,1400.0,NaN
5917,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-20,1.0,0.0,ZIPTRON,0.0,Spot,NaN,1400.0,NaN,NaN,1400.0,NaN
5918,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-12,2.0,0.0,ZIPTRON,0.0,Spot,NaN,1400.0,NaN,NaN,1400.0,NaN


In [59]:
# fill o every column where show NaN
df4.fillna(0,inplace=True)

<ipython-input-59-7fd8a8b013dc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.fillna(0,inplace=True)


In [60]:
df4

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate,current_month_day_count,per_day_rate,Plan Vehicle/Trip
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-01,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-02,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-03,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-04,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,2024-07-08,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5915,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-29,8.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0
5916,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-14,1.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0
5917,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-20,1.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0
5918,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,2024-07-12,2.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0


In [61]:
pip install simple-salesforce

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.2/132.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.2/101.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00


In [62]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [63]:
import os
import psycopg2
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import datetime
import requests
from io import StringIO
from simple_salesforce import Salesforce
from datetime import date, datetime, timedelta

def create_service(client_secret_file, api_name, api_version, scopes):
    credentials = None
    if os.path.exists(client_secret_file):
        credentials = service_account.Credentials.from_service_account_file(client_secret_file, scopes=scopes)
    else:
        print(f"The file {client_secret_file} does not exist.")
        return None

    try:
        service = build(api_name, api_version, credentials=credentials)
        print(api_name, 'service created successfully')
        return service
    except Exception as e:
        print('Unable to connect.')
        print(e)
        return None

def export_to_sheets(service, gs_sheet_id, tab_id, columns, recordset):
    my_spreadsheets = service.spreadsheets().get(spreadsheetId=gs_sheet_id).execute()

    tab_name = [sheet['properties']['title'] for sheet in my_spreadsheets['sheets'] if sheet['properties']['sheetId'] == tab_id][0]

    # Clear only columns A to D
    service.spreadsheets().values().clear(
        spreadsheetId=gs_sheet_id,
        range=f'{tab_name}!A:D'
    ).execute()

    # Insert dataset
    def construct_request_body(value_array, dimension='ROWS'):
        try:
            request_body = {
                'majorDimension': dimension,
                'values': value_array
            }
            return request_body
        except Exception as e:
            print(e)
            return {}

    # Insert column names
    request_body_columns = construct_request_body([columns])
    service.spreadsheets().values().update(
        spreadsheetId=gs_sheet_id,
        valueInputOption='USER_ENTERED',
        range=f'{tab_name}!A1',
        body=request_body_columns
    ).execute()

    # Insert rows
    request_body_values = construct_request_body(recordset)
    service.spreadsheets().values().update(
        spreadsheetId=gs_sheet_id,
        valueInputOption='USER_ENTERED',
        range=f'{tab_name}!A2',
        body=request_body_values
    ).execute()

    print(f'Data exported to {tab_name} tab')

# Salesforce connection details
sf = Salesforce(username='sagar@project-lithium.com', password='Charging987@', security_token='wyHe3xXDMDyJRiLhMPaT2XNMt')
sf_instance = 'https://project-lithium.my.salesforce.com/'
report_Id = '00OC5000000MRI7MAO'
export = '?isdtp=p1&export=1&enc=UTF-8&xf=csv'
sfUrl = sf_instance + report_Id + export
response = requests.get(sfUrl, headers=sf.headers, cookies={'sid': sf.session_id})
download_report = response.content.decode('utf-8')
df_sp = pd.read_csv(StringIO(download_report))

# Convert all date and datetime columns to string and handle NaN values
for column in df_sp.columns:
    if pd.api.types.is_datetime64_any_dtype(df_sp[column]) or pd.api.types.is_timedelta64_dtype(df_sp[column]) or pd.api.types.is_period_dtype(df_sp[column]):
        df_sp[column] = df_sp[column].astype(str)
    elif pd.api.types.is_object_dtype(df_sp[column]):
        df_sp[column] = df_sp[column].apply(lambda x: str(x) if isinstance(x, (date, datetime)) else x)

# Replace NaN values with empty strings
df_sp = df_sp.fillna('')

# Group by specified columns and count LithiumID
df_sp_new = df_sp.groupby(['Site Name', 'Attendance Date', 'City'])['LithiumID'].count().reset_index()

# Convert series to dataframe
recordset_sp = df_sp_new.reset_index().values.tolist()
columns_sp = df_sp_new.reset_index().columns.tolist()


<ipython-input-63-beab53620895>:84: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_datetime64_any_dtype(df_sp[column]) or pd.api.types.is_timedelta64_dtype(df_sp[column]) or pd.api.types.is_period_dtype(df_sp[column]):


In [64]:
df_sp_new.columns

Index(['Site Name', 'Attendance Date', 'City', 'LithiumID'], dtype='object')

In [65]:
df_sp_new

,Site Name,Attendance Date,City,LithiumID
0,,05/08/2024,Gurgaon,1
1,,10/07/2024,Gurgaon,6
2,,30/07/2024,Gurgaon,1
3,-Gurgaon-Genpact Gurgram,25/07/2024,NCR/Delhi,1
4,-Gurgaon-Genpact Gurgram,26/07/2024,NCR/Delhi,1
...,...,...,...,...
2562,ZSPune-PNQ-ZS Pune,27/07/2024,Pune,4
2563,ZSPune-PNQ-ZS Pune,28/07/2024,Pune,17
2564,ZSPune-PNQ-ZS Pune,29/07/2024,Pune,18
2565,ZSPune-PNQ-ZS Pune,30/07/2024,Pune,19


In [66]:
# show not null value column Site Name
df_sp_new=df_sp_new[df_sp_new['Site Name'].notnull()]

In [67]:
df_sp_new['Site Name']=df_sp_new['Site Name'].str.upper()

In [68]:
df_sp_new

,Site Name,Attendance Date,City,LithiumID
0,,05/08/2024,Gurgaon,1
1,,10/07/2024,Gurgaon,6
2,,30/07/2024,Gurgaon,1
3,-GURGAON-GENPACT GURGRAM,25/07/2024,NCR/Delhi,1
4,-GURGAON-GENPACT GURGRAM,26/07/2024,NCR/Delhi,1
...,...,...,...,...
2562,ZSPUNE-PNQ-ZS PUNE,27/07/2024,Pune,4
2563,ZSPUNE-PNQ-ZS PUNE,28/07/2024,Pune,17
2564,ZSPUNE-PNQ-ZS PUNE,29/07/2024,Pune,18
2565,ZSPUNE-PNQ-ZS PUNE,30/07/2024,Pune,19


In [69]:
# Convert'trip_date' into object
df4['trip_date'] = df4['trip_date'].astype(object)

<ipython-input-69-b29453f227ed>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['trip_date'] = df4['trip_date'].astype(object)


In [70]:
# Ensure 'trip_date' is in datetime64[ns] format
df4['trip_date'] = pd.to_datetime(df4['trip_date'], errors='coerce')

# Convert 'trip_date' to the desired string format
df4['trip_date'] = df4['trip_date'].dt.strftime('%d/%m/%Y')

<ipython-input-70-5689e550bca0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['trip_date'] = pd.to_datetime(df4['trip_date'], errors='coerce')
<ipython-input-70-5689e550bca0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['trip_date'] = df4['trip_date'].dt.strftime('%d/%m/%Y')


In [71]:
# Merge the dataframes
merged_df = pd.merge(df4,df_sp_new,
                     left_on=['client_office','trip_date'],
                     right_on=['Site Name','Attendance Date'],
                     how='outer')

# Ensure 'Rate' from df5 is renamed to 'Rates'
merged_df = merged_df.rename(columns={'LithiumID': 'SpCount'})

# Select the desired columns from the merged dataframe
new_df = merged_df[['client','client_office', 'billing_model', 'city_code', 'trip_date',
       'trip_count', 'escort_count', 'model', 'package_hour', 'cet_models',
       'Vehicle Count', 'Rates', 'Escort Rate', 'current_month_day_count',
       'per_day_rate','Plan Vehicle/Trip','SpCount']]


In [72]:
new_df

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate,current_month_day_count,per_day_rate,Plan Vehicle/Trip,SpCount
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,01/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,NaN
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,02/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,NaN
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,03/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,NaN
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,04/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,NaN
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,08/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
6310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0


In [73]:
# show not null value column Site Name
new_df=new_df[new_df['client_office'].notnull()]

In [74]:
new_df

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate,current_month_day_count,per_day_rate,Plan Vehicle/Trip,SpCount
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,01/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,NaN
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,02/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,NaN
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,03/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,NaN
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,04/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,NaN
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,08/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5915,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,29/07/2024,8.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,NaN
5916,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,14/07/2024,1.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,NaN
5917,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,20/07/2024,1.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,NaN
5918,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,12/07/2024,2.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,NaN


In [75]:
# Fill 0 in nan and blank cell
new_df.fillna(0,inplace=True)

<ipython-input-75-79b1734eac62>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.fillna(0,inplace=True)


In [76]:
new_df

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate,current_month_day_count,per_day_rate,Plan Vehicle/Trip,SpCount
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,01/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0.0
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,02/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0.0
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,03/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0.0
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,04/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0.0
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,08/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5915,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,29/07/2024,8.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0.0
5916,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,14/07/2024,1.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0.0
5917,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,20/07/2024,1.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0.0
5918,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,12/07/2024,2.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0.0


In [77]:
# new_df Spcount round to number dont want after . value full round value
new_df['SpCount'] = new_df['SpCount'].round(0)

<ipython-input-77-7ea39189f54f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['SpCount'] = new_df['SpCount'].round(0)


In [78]:
# take 4 column from new_df client_office,billing_model,trip_date,Vehicle Count,and last SpCount
df7=new_df[['client_office', 'billing_model', 'trip_date','Vehicle Count','SpCount']]

In [79]:
new_df

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate,current_month_day_count,per_day_rate,Plan Vehicle/Trip,SpCount
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,01/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0.0
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,02/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0.0
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,03/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0.0
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,04/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0.0
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,08/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5915,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,29/07/2024,8.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0.0
5916,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,14/07/2024,1.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0.0
5917,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,20/07/2024,1.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0.0
5918,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,12/07/2024,2.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0.0


In [80]:
import pandas as pd

df = new_df

# Function to allocate SpCount based on priority
def allocate_spcount(df):
    # Initialize dictionaries
    d24, d12, trip = {}, {}, {}

    # Populate dictionaries
    for i, row in df.iterrows():
        key = f"{row['client_office']}@{row['trip_date']}".lower()
        value = f"{int(row['Vehicle Count'])}@{int(row['SpCount'])}"

        billing_model = row['billing_model'].strip().lower()

        if billing_model == "dedicated_24":
            if key not in d24:
                d24[key] = value
        elif billing_model == "dedicated_12":
            if key not in d12:
                d12[key] = value
        elif billing_model == "trip":
            if key not in trip:
                trip[key] = value

    # Update values in d24
    for key, value in d24.items():
        vcount, spcount = map(int, value.split('@'))
        actcount = spcount if not (d12.get(key) or trip.get(key)) else min(spcount, vcount * 2)
        d24[key] = f"{vcount}@{spcount}@{actcount if spcount != 0 else 0}"

    # Update values in d12
    for key, value in d12.items():
        vcount, spcount = map(int, value.split('@'))
        actcount = spcount
        if not (d24.get(key) or trip.get(key)):
            actcount = spcount
        elif d24.get(key) and trip.get(key):
            d24vcount, d24spcount, d24actcount = map(int, d24[key].split('@'))
            actcount = max(0, spcount - d24actcount) if d24vcount > vcount else vcount
        elif d24.get(key):
            d24spcount, d24actcount = map(int, d24[key].split('@')[1:])
            actcount = max(0, d24spcount - d24actcount) if d24spcount != d24actcount else 0
        elif trip.get(key):
            actcount = min(spcount, vcount)
        d12[key] = f"{vcount}@{spcount}@{actcount if spcount != 0 else 0}"

    # Update values in trip
    for key, value in trip.items():
        vcount, spcount = map(int, value.split('@'))
        actcount = spcount
        if d24.get(key):
            actcount -= int(d24[key].split('@')[2])
        if d12.get(key):
            actcount -= int(d12[key].split('@')[2])
        trip[key] = f"{vcount}@{spcount}@{max(0, actcount if spcount != 0 else 0)}"

    # Add actcount column to dataframe
    df['actcount'] = 0  # Initialize the column

    for i, row in df.iterrows():
        key = f"{row['client_office']}@{row['trip_date']}".lower()
        billing_model = row['billing_model'].strip().lower()

        if billing_model == "dedicated_24":
            df.at[i, 'actcount'] = int(d24[key].split('@')[2])
        elif billing_model == "dedicated_12":
            if key in d12:
                df.at[i, 'actcount'] = int(d12[key].split('@')[2])
        elif billing_model == "trip":
            if key in trip:
                df.at[i, 'actcount'] = int(trip[key].split('@')[2])

    return df

# Apply the function to allocate SpCount
df = allocate_spcount(df)

# Display the updated DataFrame
print(df)


<ipython-input-80-635f4a90b123>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['actcount'] = 0  # Initialize the column


            client              client_office billing_model city_code  \
0       HULBELGAUM  HULBELGAUM-BGM-HULBELGAUM  DEDICATED_12       BGM   
1       HULBELGAUM  HULBELGAUM-BGM-HULBELGAUM  DEDICATED_12       BGM   
2       HULBELGAUM  HULBELGAUM-BGM-HULBELGAUM  DEDICATED_12       BGM   
3       HULBELGAUM  HULBELGAUM-BGM-HULBELGAUM  DEDICATED_12       BGM   
4       HULBELGAUM  HULBELGAUM-BGM-HULBELGAUM  DEDICATED_12       BGM   
...            ...                        ...           ...       ...   
5915  Chennai-Spot               CHENNAI-SPOT          SPOT       CHN   
5916  Chennai-Spot               CHENNAI-SPOT          SPOT       CHN   
5917  Chennai-Spot               CHENNAI-SPOT          SPOT       CHN   
5918  Chennai-Spot               CHENNAI-SPOT          SPOT       CHN   
5919  Chennai-Spot               CHENNAI-SPOT          SPOT       CHN   

       trip_date  trip_count  escort_count    model  package_hour cet_models  \
0     01/07/2024         2.0           0.0 

In [81]:
df

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate,current_month_day_count,per_day_rate,Plan Vehicle/Trip,SpCount,actcount
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,01/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0.0,0
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,02/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0.0,0
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,03/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0.0,0
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,04/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0.0,0
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,08/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5915,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,29/07/2024,8.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0.0,0
5916,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,14/07/2024,1.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0.0,0
5917,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,20/07/2024,1.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0.0,0
5918,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,12/07/2024,2.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0.0,0


In [82]:
# drop column SpCount
df.drop(columns=['SpCount'],inplace=True)

<ipython-input-82-e8a768977b2c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['SpCount'],inplace=True)


In [83]:
df

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate,current_month_day_count,per_day_rate,Plan Vehicle/Trip,actcount
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,01/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,02/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,03/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,04/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,08/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5915,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,29/07/2024,8.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0
5916,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,14/07/2024,1.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0
5917,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,20/07/2024,1.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0
5918,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,12/07/2024,2.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0


In [84]:
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 54.9 MB/s eta 0:00:00


In [85]:
df.columns

Index(['client', 'client_office', 'billing_model', 'city_code', 'trip_date',
       'trip_count', 'escort_count', 'model', 'package_hour', 'cet_models',
       'Vehicle Count', 'Rates', 'Escort Rate', 'current_month_day_count',
       'per_day_rate', 'Plan Vehicle/Trip', 'actcount'],
      dtype='object')

In [86]:
#create two column SM and RM in df
df['SM'] = 0
df['RM'] = 0
df['Contracted_vehicle']=0

<ipython-input-86-073158bc9df9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SM'] = 0
<ipython-input-86-073158bc9df9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RM'] = 0
<ipython-input-86-073158bc9df9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [87]:
df

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate,current_month_day_count,per_day_rate,Plan Vehicle/Trip,actcount,SM,RM,Contracted_vehicle
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,01/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0,0,0,0
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,02/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0,0,0,0
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,03/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0,0,0,0
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,04/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0,0,0,0
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,08/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5915,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,29/07/2024,8.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0,0,0,0
5916,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,14/07/2024,1.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0,0,0,0
5917,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,20/07/2024,1.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0,0,0,0
5918,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,12/07/2024,2.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0,0,0,0


In [88]:
def update_sm_rm(df):
    for index, row in df.iterrows():
        city_code = row['city_code']
        static_value = f"UNDER_{city_code}"
        df.at[index, 'SM'] = static_value
        df.at[index, 'RM'] = static_value
    return df

In [89]:
df_ok = update_sm_rm(df)

<ipython-input-88-1f6003660cd3>:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'UNDER_BGM' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'SM'] = static_value
<ipython-input-88-1f6003660cd3>:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'UNDER_BGM' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'RM'] = static_value


In [90]:
df_ok

,client,client_office,billing_model,city_code,trip_date,trip_count,escort_count,model,package_hour,cet_models,Vehicle Count,Rates,Escort Rate,current_month_day_count,per_day_rate,Plan Vehicle/Trip,actcount,SM,RM,Contracted_vehicle
0,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,01/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0,UNDER_BGM,UNDER_BGM,0
1,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,02/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0,UNDER_BGM,UNDER_BGM,0
2,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,03/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0,UNDER_BGM,UNDER_BGM,0
3,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,04/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0,UNDER_BGM,UNDER_BGM,0
4,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,BGM,08/07/2024,2.0,0.0,0,0.0,CET,1.0,65000.0,0,23.0,2826.0,1,0,UNDER_BGM,UNDER_BGM,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5915,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,29/07/2024,8.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0,UNDER_CHN,UNDER_CHN,0
5916,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,14/07/2024,1.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0,UNDER_CHN,UNDER_CHN,0
5917,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,20/07/2024,1.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0,UNDER_CHN,UNDER_CHN,0
5918,Chennai-Spot,CHENNAI-SPOT,SPOT,CHN,12/07/2024,2.0,0.0,ZIPTRON,0.0,Spot,0.0,1400.0,0,0.0,1400.0,0,0,UNDER_CHN,UNDER_CHN,0


In [91]:
df_ok.columns

Index(['client', 'client_office', 'billing_model', 'city_code', 'trip_date',
       'trip_count', 'escort_count', 'model', 'package_hour', 'cet_models',
       'Vehicle Count', 'Rates', 'Escort Rate', 'current_month_day_count',
       'per_day_rate', 'Plan Vehicle/Trip', 'actcount', 'SM', 'RM',
       'Contracted_vehicle'],
      dtype='object')

In [92]:
# Desired column order
new_order = ['city_code', 'client', 'client_office','billing_model','trip_date','trip_count','escort_count','Rates','current_month_day_count','per_day_rate','SM','RM','Plan Vehicle/Trip','Vehicle Count','actcount','Escort Rate','Contracted_vehicle','model','package_hour','cet_models']

In [93]:
df = df[new_order]

In [94]:
df

,city_code,client,client_office,billing_model,trip_date,trip_count,escort_count,Rates,current_month_day_count,per_day_rate,SM,RM,Plan Vehicle/Trip,Vehicle Count,actcount,Escort Rate,Contracted_vehicle,model,package_hour,cet_models
0,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,01/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1,1.0,0,0,0,0,0.0,CET
1,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,02/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1,1.0,0,0,0,0,0.0,CET
2,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,03/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1,1.0,0,0,0,0,0.0,CET
3,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,04/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1,1.0,0,0,0,0,0.0,CET
4,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,08/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1,1.0,0,0,0,0,0.0,CET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5915,CHN,Chennai-Spot,CHENNAI-SPOT,SPOT,29/07/2024,8.0,0.0,1400.0,0.0,1400.0,UNDER_CHN,UNDER_CHN,0,0.0,0,0,0,ZIPTRON,0.0,Spot
5916,CHN,Chennai-Spot,CHENNAI-SPOT,SPOT,14/07/2024,1.0,0.0,1400.0,0.0,1400.0,UNDER_CHN,UNDER_CHN,0,0.0,0,0,0,ZIPTRON,0.0,Spot
5917,CHN,Chennai-Spot,CHENNAI-SPOT,SPOT,20/07/2024,1.0,0.0,1400.0,0.0,1400.0,UNDER_CHN,UNDER_CHN,0,0.0,0,0,0,ZIPTRON,0.0,Spot
5918,CHN,Chennai-Spot,CHENNAI-SPOT,SPOT,12/07/2024,2.0,0.0,1400.0,0.0,1400.0,UNDER_CHN,UNDER_CHN,0,0.0,0,0,0,ZIPTRON,0.0,Spot


In [95]:
df.columns

Index(['city_code', 'client', 'client_office', 'billing_model', 'trip_date',
       'trip_count', 'escort_count', 'Rates', 'current_month_day_count',
       'per_day_rate', 'SM', 'RM', 'Plan Vehicle/Trip', 'Vehicle Count',
       'actcount', 'Escort Rate', 'Contracted_vehicle', 'model',
       'package_hour', 'cet_models'],
      dtype='object')

In [96]:
# rename colum name multiple
df.rename(columns={'escort_count':'escort_trip_count'},inplace=True)
df.rename(columns={'Rates':'Rate'},inplace=True)
df.rename(columns={'per_day_rate':'Perdayrate'},inplace=True)
df.rename(columns={'current_month_day_count':'Operation_Days'},inplace=True)
df.rename(columns={'actcount':'Sp'},inplace=True)
df.rename(columns={'Escort Rate':'Escort_rate'},inplace=True)
df.rename(columns={'model':'Veh Type'},inplace=True)
df.rename(columns={'package_hour':'Package_Hour'},inplace=True)
df.rename(columns={'Vehicle Count':'Vehicle_mapping'},inplace=True)
df.rename(columns={'Plan Vehicle/Trip':'Plan_Vehicle_Trip'},inplace=True)
df.rename(columns={'Veh Type':'Veh_Type'},inplace=True)

In [97]:
#fill 0 to all blank
df.fillna(0,inplace=True)

In [98]:
df

,city_code,client,client_office,billing_model,trip_date,trip_count,escort_trip_count,Rate,Operation_Days,Perdayrate,SM,RM,Plan_Vehicle_Trip,Vehicle_mapping,Sp,Escort_rate,Contracted_vehicle,Veh_Type,Package_Hour,cet_models
0,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,01/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1,1.0,0,0,0,0,0.0,CET
1,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,02/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1,1.0,0,0,0,0,0.0,CET
2,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,03/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1,1.0,0,0,0,0,0.0,CET
3,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,04/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1,1.0,0,0,0,0,0.0,CET
4,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,08/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1,1.0,0,0,0,0,0.0,CET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5915,CHN,Chennai-Spot,CHENNAI-SPOT,SPOT,29/07/2024,8.0,0.0,1400.0,0.0,1400.0,UNDER_CHN,UNDER_CHN,0,0.0,0,0,0,ZIPTRON,0.0,Spot
5916,CHN,Chennai-Spot,CHENNAI-SPOT,SPOT,14/07/2024,1.0,0.0,1400.0,0.0,1400.0,UNDER_CHN,UNDER_CHN,0,0.0,0,0,0,ZIPTRON,0.0,Spot
5917,CHN,Chennai-Spot,CHENNAI-SPOT,SPOT,20/07/2024,1.0,0.0,1400.0,0.0,1400.0,UNDER_CHN,UNDER_CHN,0,0.0,0,0,0,ZIPTRON,0.0,Spot
5918,CHN,Chennai-Spot,CHENNAI-SPOT,SPOT,12/07/2024,2.0,0.0,1400.0,0.0,1400.0,UNDER_CHN,UNDER_CHN,0,0.0,0,0,0,ZIPTRON,0.0,Spot


In [99]:
df.columns

Index(['city_code', 'client', 'client_office', 'billing_model', 'trip_date',
       'trip_count', 'escort_trip_count', 'Rate', 'Operation_Days',
       'Perdayrate', 'SM', 'RM', 'Plan_Vehicle_Trip', 'Vehicle_mapping', 'Sp',
       'Escort_rate', 'Contracted_vehicle', 'Veh_Type', 'Package_Hour',
       'cet_models'],
      dtype='object')

In [100]:
# fill 0 if blank in column Escort_rate
df['Escort_rate'].fillna(0,inplace=True)

In [101]:
df

,city_code,client,client_office,billing_model,trip_date,trip_count,escort_trip_count,Rate,Operation_Days,Perdayrate,SM,RM,Plan_Vehicle_Trip,Vehicle_mapping,Sp,Escort_rate,Contracted_vehicle,Veh_Type,Package_Hour,cet_models
0,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,01/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1,1.0,0,0,0,0,0.0,CET
1,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,02/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1,1.0,0,0,0,0,0.0,CET
2,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,03/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1,1.0,0,0,0,0,0.0,CET
3,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,04/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1,1.0,0,0,0,0,0.0,CET
4,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,08/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1,1.0,0,0,0,0,0.0,CET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5915,CHN,Chennai-Spot,CHENNAI-SPOT,SPOT,29/07/2024,8.0,0.0,1400.0,0.0,1400.0,UNDER_CHN,UNDER_CHN,0,0.0,0,0,0,ZIPTRON,0.0,Spot
5916,CHN,Chennai-Spot,CHENNAI-SPOT,SPOT,14/07/2024,1.0,0.0,1400.0,0.0,1400.0,UNDER_CHN,UNDER_CHN,0,0.0,0,0,0,ZIPTRON,0.0,Spot
5917,CHN,Chennai-Spot,CHENNAI-SPOT,SPOT,20/07/2024,1.0,0.0,1400.0,0.0,1400.0,UNDER_CHN,UNDER_CHN,0,0.0,0,0,0,ZIPTRON,0.0,Spot
5918,CHN,Chennai-Spot,CHENNAI-SPOT,SPOT,12/07/2024,2.0,0.0,1400.0,0.0,1400.0,UNDER_CHN,UNDER_CHN,0,0.0,0,0,0,ZIPTRON,0.0,Spot


In [102]:
# Step 1: Install required libraries
!pip install psycopg2-binary pandas

In [103]:
# export data
df.to_csv('df.csv')

In [104]:
# Convert the column to numeric, with errors coerced to NaN
df['Escort_rate'] = pd.to_numeric(df['Escort_rate'], errors='coerce')
df['Plan_Vehicle_Trip'] = pd.to_numeric(df['Plan_Vehicle_Trip'], errors='coerce')

# Fill NaN values with 0
df['Escort_rate'].fillna(0, inplace=True)
df['Plan_Vehicle_Trip'].fillna(0, inplace=True)

In [105]:
df

,city_code,client,client_office,billing_model,trip_date,trip_count,escort_trip_count,Rate,Operation_Days,Perdayrate,SM,RM,Plan_Vehicle_Trip,Vehicle_mapping,Sp,Escort_rate,Contracted_vehicle,Veh_Type,Package_Hour,cet_models
0,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,01/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1.0,1.0,0,0,0,0,0.0,CET
1,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,02/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1.0,1.0,0,0,0,0,0.0,CET
2,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,03/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1.0,1.0,0,0,0,0,0.0,CET
3,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,04/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1.0,1.0,0,0,0,0,0.0,CET
4,BGM,HULBELGAUM,HULBELGAUM-BGM-HULBELGAUM,DEDICATED_12,08/07/2024,2.0,0.0,65000.0,23.0,2826.0,UNDER_BGM,UNDER_BGM,1.0,1.0,0,0,0,0,0.0,CET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5915,CHN,Chennai-Spot,CHENNAI-SPOT,SPOT,29/07/2024,8.0,0.0,1400.0,0.0,1400.0,UNDER_CHN,UNDER_CHN,0.0,0.0,0,0,0,ZIPTRON,0.0,Spot
5916,CHN,Chennai-Spot,CHENNAI-SPOT,SPOT,14/07/2024,1.0,0.0,1400.0,0.0,1400.0,UNDER_CHN,UNDER_CHN,0.0,0.0,0,0,0,ZIPTRON,0.0,Spot
5917,CHN,Chennai-Spot,CHENNAI-SPOT,SPOT,20/07/2024,1.0,0.0,1400.0,0.0,1400.0,UNDER_CHN,UNDER_CHN,0.0,0.0,0,0,0,ZIPTRON,0.0,Spot
5918,CHN,Chennai-Spot,CHENNAI-SPOT,SPOT,12/07/2024,2.0,0.0,1400.0,0.0,1400.0,UNDER_CHN,UNDER_CHN,0.0,0.0,0,0,0,ZIPTRON,0.0,Spot


In [106]:
print(df['Escort_rate'].dtype)

int64


In [107]:
# export data
df.to_csv('df.csv')

In [108]:
import pandas as pd
import psycopg2
import logging

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Check for column names and rename if necessary
df.rename(columns={
    'Operation Days': 'Operation_Days',
    'Package Hour': 'Package_Hour'
}, inplace=True)

# Verify columns after renaming
print("DataFrame Columns after renaming:", df.columns)

# Convert date columns to the format PostgreSQL expects
df.loc[:, 'trip_date'] = pd.to_datetime(df['trip_date'], format='%d/%m/%Y', errors='coerce').dt.strftime('%Y-%m-%d')

# Convert columns to numeric, replace errors with NaN, and then fill NaN with default values (0 in this case)
numeric_columns = ['Rate', 'Operation_Days', 'Perdayrate', 'Escort_rate', 'Contracted_vehicle', 'Package_Hour']
df.loc[:, numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

# Define the batch size
batch_size = 1000

# Connect to the PostgreSQL database
try:
    connection = psycopg2.connect(
        dbname="postgres",
        user="tableau",
        password="tableauTeam@123",
        host="34.100.223.97",
        port="5432"
    )
    cursor = connection.cursor()
    logging.info("Connected to the database successfully")

    # Truncate the table before inserting new data
    try:
        cursor.execute("TRUNCATE TABLE mrr")
        connection.commit()
        logging.info("Table 'mrr' truncated successfully")
    except Exception as e:
        logging.error(f"Error truncating table: {e}")
        connection.rollback()  # Rollback the transaction if there's an error

    # Insert data into the database in chunks
    for start in range(0, len(df), batch_size):
        end = min(start + batch_size, len(df))
        batch = df.iloc[start:end]

        # Filter out rows where 'trip_date' is NaN
        batch = batch.dropna(subset=['trip_date'])

        # Create a list of tuples for batch insertion
        data_tuples = [
            (
                row['city_code'], row['client'], row['client_office'], row['billing_model'], row['trip_date'],
                row['trip_count'], row['escort_trip_count'], row['Rate'], row['Operation_Days'], row['Perdayrate'],
                row['SM'], row['RM'], row['Plan_Vehicle_Trip'], row['Vehicle_mapping'], row['Sp'], row['Escort_rate'],
                row['Contracted_vehicle'], row['Veh_Type'], row['Package_Hour'], row['cet_models']
            )
            for _, row in batch.iterrows()
        ]

        # Log the first few tuples to verify the data structure
        logging.info(f"First few tuples to be inserted: {data_tuples[:5]}")

        # Execute batch insertion
        try:
            cursor.executemany(
                """
                INSERT INTO mrr (
                    city_code, client, client_office, billing_model, trip_date, trip_count,
                    escort_trip_count, Rate, Operation_Days, Perdayrate, SM, RM,
                    Plan_Vehicle_Trip, Vehicle_mapping, Sp, Escort_rate, Contracted_vehicle,
                    Veh_Type, Package_Hour,cet_models
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s)
                """,
                data_tuples
            )
            connection.commit()  # Ensure the batch is committed
            logging.info(f"Inserted rows {start + 1} to {end}")
        except Exception as e:
            logging.error(f"Error inserting batch {start + 1} to {end}: {e}")
            connection.rollback()  # Rollback the transaction if there's an error

    # Verify the data count
    cursor.execute("SELECT COUNT(*) FROM mrr")
    count = cursor.fetchone()[0]
    logging.info(f"Total rows in mrr: {count}")

except Exception as e:
    logging.error(f"Error: {e}")
finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()
    logging.info("Connection closed")


DataFrame Columns after renaming: Index(['city_code', 'client', 'client_office', 'billing_model', 'trip_date',
       'trip_count', 'escort_trip_count', 'Rate', 'Operation_Days',
       'Perdayrate', 'SM', 'RM', 'Plan_Vehicle_Trip', 'Vehicle_mapping', 'Sp',
       'Escort_rate', 'Contracted_vehicle', 'Veh_Type', 'Package_Hour',
       'cet_models'],
      dtype='object')
